In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer

/home/kappa/youtube-video-indexer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data = pd.read_parquet("../data/transcripts.parquet", dtype_backend="pyarrow")
data = data.fillna("")

In [5]:
# Good trade-off between speed and accuracy
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [6]:
text_vector = embedder.encode(
    data["text"].tolist(), show_progress_bar=True, batch_size=32
)

Batches: 100%|██████████| 42/42 [00:16<00:00,  2.56it/s]


In [7]:
title_vector = embedder.encode(
    data["title"].tolist(), show_progress_bar=True, batch_size=32
)

Batches: 100%|██████████| 42/42 [00:00<00:00, 95.14it/s] 


In [8]:
data.loc[:, "text_embedding"] = pd.DataFrame({"a": list(text_vector)})
data.loc[:, "title_embedding"] = pd.DataFrame({"b": list(title_vector)})

In [9]:
data[
    [
        "id",
        "title_embedding",
        "text_embedding",
    ]
].to_parquet("../data/transcripts_with_embeddings.parquet", index=False)